# From Prev.
Rearranged code
Used larger training set

In [9]:
import pandas as pd
from urllib.parse import unquote

df = pd.read_json('data.json',encoding='utf-8')
#df.drop(['date','body','author','url'],axis=1,inplace=True) #for test_2.json
#df.drop(['id','news_content','date','url'],axis=1,inplace=True)

In [10]:
df['category'] = df['category'].apply(unquote)

In [11]:
df['category'].unique()

array(['politics', 'ข่าวด่วน', 'ข่าวการเมือง', 'covid-19', 'economics',
       'breaking-news', 'in-the-news', 'special-stories', 'politic',
       'local', 'society', 'entertainment', 'abroad', 'ข่าวต่างประเทศ',
       'index.php', 'ข่าวกีฬา', 'economy', 'bbc-thai',
       'around-the-world-news', 'pr-news', 'sports', 'trcheerthai',
       'business', 'thaifootball', 'foreign', 'eurofootball', 'others',
       'news', 'crime', 'ข่าวในประเทศ', 'ข่าวเศรษฐกิจ', 'blunt-opinion',
       'around-thailand', 'hot-topics', 'food', 'advertorial',
       'uncategorized', 'belief', 'woman', 'khrongkhaidaasapparod',
       'sport', 'm-market', 'lifestyle', 'monitor-news', 'life', 'auto',
       '1293427', 'volleyball', 'tech', 'futsal', 'ข่าวบันเทิง',
       'house-condo', 'หวย', 'tv-guide', 'boxing-mma',
       'analysis-today-politics', 'ดวง', 'online-social', 'hot-clips',
       'game-esport', '1555365'], dtype=object)

In [5]:
import numpy as np
X = np.array(df['news_topic'])
y = np.array(df['category'])

In [27]:
import re
#prepare each sentence
def text_prepare(text):
    text = re.sub("[/(){}\[\]\|@,;\n\t:\'\- “”]", '', text)
    text = re.sub("[/\xa0]", '', text)
    text = text.replace('"','').replace("'",'')
    return text

In [ ]:
#test text_prepare()
text_prepare(df.iloc[23]['news_topic'])

In [29]:
#execute prepare
X = np.array([text_prepare(title) for title in X])

In [30]:
X

array(['เปิดใจอนุทินชาญวีรกูลหัวหน้าพรรคภูมิใจไทยปมเก้าอี้รัฐมนตรีใช้ดีลเดิม',
       'อนุทินเตรียมผลักดันสนามบินสตูลและสะพานถนนสตูลเปอร์ลิส',
       'อภิสิทธิ์\u200bเผยเพิ่งคุยไลน์ขำๆกับอนุทิน\u200bเรื่อง\u200bกระแสข่าวณัฏฐพลเผยมีพรรคร่วมรัฐบาลแล้ว20พรรค',
       'อนุทินเผยนายกฯบรรจุกัญชาเป็นนโยบายรัฐบาลแล้ว',
       'อนุทินไม่ติดใจลงโทษสิริพงษ์โหวตสวนมติพรรค',
       'อนุทินโพสต์ภาพนั่งไหว้พระบอกชาติศาสนาพระมหากษัตริย์ประชาชนไปทางนี้แหละไม่มีหลงทาง',
       'อนุทินพร้อมเป็นฝ่ายค้านถ้ารัฐบาลไม่เอากัญชา',
       'อนุทินโต้ข่าวลือยันยังไม่จับมือขั้วพปชร.ขอคิดรอบคอบหากใครยังไม่พอใจครั้งหน้าไม่ต้องเลือก',
       'อนุทินลั่นได้ปลูกกัญชาบ้านละ6ต้นแน่',
       'อนุทินส่งรายชื่อครม.ถึงมือนายกฯพรุ่งนี้',
       'ภูมิใจไทยประชุมใหญ่จ่อเลือกอนุทินนั่งหัวหน้าพรรคต่อ',
       'อนุทินเรียกส.ส.ศรีสะเกษยกมือสวนมติพรรคงดออกเสียงเลือกประยุทธ์เป็นนายกฯ',
       'มติภูมิใจไทยให้อนุทินประสานตั้งรัฐบาลยันอยู่ฝ่ายเสียงข้างมาก',
       'ภูมิใจไทยเห็นตรงกันชูอนุทินบัญชีนายกฯเบอร์เดียว',
       'อนุทินโต้เรือง

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [32]:
type(X_train)

numpy.ndarray

In [33]:
len(X_test)

205

### Bag of words

In [34]:
from pythainlp.tokenize import word_tokenize

In [35]:

#tokenize function
def Thaitokenize(text):
    """
    turn a Thai sentence into Thai words with meaning
    """
    text = ''.join(text.split('"')) ##handled in regex??
    text = word_tokenize(text , engine = 'pyicu')
    return text

In [27]:
#try mapping function
#Error
#list(map(Thaitokenize,X_train))

In [36]:
Thaitokenize('อนุทินแย้มข่าวดีกลางสภา!ไทยเตรียมออกจากการใช้พ.ร.ก.ฉุกเฉินStandards of Conduct for Business\xa0')

['อนุทิน',
 'แย้ม',
 'ข่าวดี',
 'กลาง',
 'สภา',
 '!',
 'ไทย',
 'เตรียม',
 'ออกจาก',
 'การ',
 'ใช้',
 'พ.ร.ก.',
 'ฉุกเฉิน',
 'Standards',
 ' ',
 'of',
 ' ',
 'Conduct',
 ' ',
 'for',
 ' ',
 'Business',
 '\xa0']

### just a test for fun###
%timeit word_tokenize('แสนสิริสร้างมาตรฐานใหม่ของการยอมรับความเท่าเทียม' , engine = 'pyicu')

### just a test for fun###
%timeit word_tokenize('แสนสิริสร้างมาตรฐานใหม่ของการยอมรับความเท่าเทียม' , engine = 'newmm')

In [37]:
#words to account in the bag
words_counts = {}

for i in X_train: #each i is a topic sentence
    wordlist = Thaitokenize(i)
    for j in wordlist:
        words_counts[j] = words_counts.get(j,0) + 1
#words_counts = 238

WORDS_TO_INDEX = dict(zip(words_counts.keys(),list(x for x in range(len(words_counts)))))

In [38]:
len(words_counts)

3106

In [39]:
#create a bag of words for a sentence
def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a sentence
        words_to_index: words to count in the bag 
        e.g.
        words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
        
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    result_vector = np.zeros(dict_size)
    
    #tokenize Thai words
    words = Thaitokenize(text)
    
    for i in words:
      if(words_to_index.get(i,-1)!=-1) :
        result_vector[words_to_index[i]]=+1
    return result_vector

In [40]:
X_train_my_bag = np.array([my_bag_of_words(text, WORDS_TO_INDEX, len(words_counts)) for text in X_train])

In [33]:
# [word for word in WORDS_TO_INDEX]
type(WORDS_TO_INDEX.keys())

dict_keys

In [41]:
#for illustration & checking
myBagX = pd.DataFrame(X_train_my_bag,columns = WORDS_TO_INDEX.keys())
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
#myBagX

In [37]:
X_train

array(['จับพิรุธเรตติ้งละครดร.เสรีกล้าพูด!สำรวจน้อยไปหรือเปล่า?',
       'จินตหราเปิดใจทั้งน้ำตาเล่าลางสังหรณ์ก่อนสูญเสียคุณแม่',
       'หมอแนะวิธีเลี้ยงลูกไม่ให้โตขึ้นไปฆ่าใครครอบครัวมีส่วนช่วยได้',
       'รอลุ้นแชมป์TheGoldenSongเวทีเพลงเพราะใครจะคว้าไปครองเป็นคนแรก',
       '"มูรินโญ"ยืนกรานชัดไม่มีแผนเสริมทัพไก่ช่วงหน้าหนาว',
       'เชียงใหม่ส่งยาบ้าทางพัสดุไปรษณีย์จับได้กว่า3หมื่นเม็ด',
       'อนุทินพร้อมเป็นฝ่ายค้านถ้ารัฐบาลไม่เอากัญชา',
       'อ๊อฟปองศักดิ์สูญเสียคุณแม่ด้วยโรคมะเร็งหลังต่อสู้ด้วยกันมา10ปี',
       'กันนภัทรง้อมารีสำเร็จเริ่มต้นความหวานกลับมาลงรูปด้วยกันอีกครั้ง',
       'ไทยเกาหลีลงนามบันทึกความเข้าใจระหว่างศูนย์วัฒนธรรมอาเซียนณกรุงเทพฯปูซานเดินหน้าแลกเปลี่ยนมรดกวัฒนธรรมทั้งที่จับต้องได้จับต้องไม่ได้อุตสาหกรรมเชิงสร้างสรรค์กิจกรรมศิลปวัฒนธรรมประเดิมงานแรกโครงการวัฒนธรรมสัญจรอาเซียนสู่โลกที่เกาหลีตุลาคมนี้',
       'แท็กซี่น้ำใจงามเก็บเงินเกือบ3แสนส่งคืนผู้โดยสาร',
       'มท.2แย้มเลือกตั้งท้องถิ่นหลังก.พ.63ยังไม่เคาะ"กทม.พัทยา"ใครก่อน',
       '"แอนนานา"ร่ว

In [179]:
#turn y values to 1s,0s

In [211]:
# from sklearn.preprocessing import MultiLabelBinarizer
# this is for list of tags

In [42]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
#lb.fit_transform([1, 2, 6, 4, 2]) 
#fit & transform in one step
#fit = figure out different classes and turn into necessary binary formats
#transform = transform given data into binary formats

In [43]:
y_train_lb = lb.fit_transform(y_train)


In [44]:
lb.classes_.size

56

In [45]:
#for illustration & checking
Y_LB = pd.DataFrame(y_train_lb,columns = lb.classes_)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
Y_LB.iloc[10:20]
#y_train.iloc[10:20]

,1293427,abroad,advertorial,analysis-today-politics,around-thailand,around-the-world-news,auto,bbc-thai,belief,breaking-news,business,covid-19,crime,economics,economy,entertainment,eurofootball,food,foreign,futsal,hot-clips,hot-topics,house-condo,in-the-news,index.php,khrongkhaidaasapparod,life,lifestyle,local,monitor-news,news,online-social,others,politic,politics,pr-news,society,special-stories,sport,sports,tech,thaifootball,trcheerthai,tv-guide,uncategorized,volleyball,woman,ข่าวการเมือง,ข่าวกีฬา,ข่าวด่วน,ข่าวต่างประเทศ,ข่าวบันเทิง,ข่าวเศรษฐกิจ,ข่าวในประเทศ,ดวง,หวย
10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
y_train_lb.shape

(614, 56)

In [47]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
#example
X = np.array([
     [10, 10],
     [8, 10],
     [-5, 5.5],
     [-5.4, 5.5],
     [-20, -20],
     [-15, -20]])
y = np.array([0, 0, 1, 1, 2, 2])
clf = OneVsRestClassifier(SVC()).fit(X, y)
clf.predict([[-19, -20], [9, 9], [-5, 5]])

array([2, 0, 1])

In [53]:
clf = OneVsRestClassifier(SVC()).fit(X_train_my_bag, y_train_lb)

In [54]:
#change label arrays back to words
lb.inverse_transform(y_train_lb)

array(['news', 'news', 'society', 'news', 'eurofootball', 'local',
       'ข่าวการเมือง', 'news', 'news', 'pr-news', 'local', 'politic',
       'news', 'special-stories', 'news', 'politic', 'entertainment',
       'local', 'local', 'politics', 'business', 'entertainment',
       'lifestyle', 'politics', 'news', 'local', 'politic', 'local',
       'economics', 'หวย', 'local', 'trcheerthai', 'local', 'bbc-thai',
       'entertainment', 'local', 'foreign', 'entertainment', 'woman',
       'advertorial', 'crime', 'eurofootball', 'society', 'ข่าวบันเทิง',
       'politics', 'crime', 'news', 'sport', 'khrongkhaidaasapparod',
       'local', 'politics', 'politic', 'around-the-world-news',
       'entertainment', 'entertainment', 'ข่าวด่วน', 'politic',
       'entertainment', 'society', 'entertainment', 'entertainment',
       'news', 'local', 'covid-19', 'around-thailand',
       'khrongkhaidaasapparod', 'foreign', 'local', 'local',
       'entertainment', 'hot-clips', 'ข่าวเศรษฐกิจ', 'entert

In [60]:
#check the predictions, models applied correctly? Yes
y = clf.predict(X_train_my_bag)
lb.inverse_transform(y)
#y = lb.inverse_transform(y)
#print(np.concatenate((y.reshape(len(y),1), y_train.reshape(len(y_train),1)),1))

array(['1293427', 'news', '1293427', 'news', '1293427', 'local',
       '1293427', 'news', '1293427', 'pr-news', 'local', 'politic',
       'news', '1293427', '1293427', 'politic', 'entertainment',
       '1293427', 'local', 'politics', '1293427', 'entertainment',
       '1293427', 'politics', 'news', '1293427', '1293427', 'local',
       'economics', '1293427', '1293427', '1293427', 'local', 'bbc-thai',
       'entertainment', 'local', '1293427', 'entertainment', '1293427',
       'advertorial', '1293427', 'eurofootball', '1293427', '1293427',
       'politics', '1293427', 'news', '1293427', '1293427', 'local',
       'politics', 'politic', '1293427', 'entertainment', '1293427',
       '1293427', 'politic', 'entertainment', 'society', '1293427',
       'entertainment', 'news', '1293427', '1293427', '1293427',
       '1293427', '1293427', 'local', 'local', '1293427', '1293427',
       '1293427', '1293427', 'entertainment', '1293427', '1293427',
       '1293427', '1293427', 'local', 'po

In [57]:
X_test_my_bag = np.array([my_bag_of_words(text, WORDS_TO_INDEX, len(words_counts)) for text in X_test])
y_pred = lb.inverse_transform(clf.predict(X_test_my_bag))
y_test = np.array(y_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
#print(np.concatenate(y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1),1))

[['1293427' 'society']
 ['1293427' 'politics']
 ['1293427' 'politic']
 ['1293427' 'news']
 ['1293427' 'ข่าวต่างประเทศ']
 ['1293427' 'ข่าวด่วน']
 ['1293427' 'economics']
 ['1293427' 'pr-news']
 ['1293427' 'entertainment']
 ['1293427' 'news']
 ['1293427' 'news']
 ['1293427' 'politic']
 ['1293427' 'politics']
 ['1293427' 'breaking-news']
 ['1293427' 'politic']
 ['1293427' 'entertainment']
 ['1293427' 'news']
 ['1293427' 'local']
 ['1293427' 'entertainment']
 ['1293427' 'politic']
 ['1293427' 'entertainment']
 ['1293427' 'local']
 ['1293427' 'local']
 ['1293427' 'entertainment']
 ['1293427' 'around-thailand']
 ['1293427' 'entertainment']
 ['1293427' 'crime']
 ['1293427' 'politic']
 ['1293427' 'society']
 ['1293427' 'entertainment']
 ['1293427' 'politic']
 ['1293427' 'politic']
 ['1293427' 'ข่าวการเมือง']
 ['1293427' 'm-market']
 ['1293427' 'politic']
 ['1293427' 'local']
 ['1293427' 'entertainment']
 ['1293427' 'society']
 ['1293427' 'news']
 ['1293427' 'local']
 ['1293427' 'news']
 ['1293

In [58]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [2 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


0.0

In [52]:
test = text_prepare('“อนุทิน” ขอ ผู้ร้อง เปิดชื่อ อักษรย่อ \"พ\" คนหักหัวคิวโรงแรม 40%')
test = np.array([my_bag_of_words(test, WORDS_TO_INDEX, len(words_counts))])
clf.predict(test)

array(['politic'], dtype='<U23')